In [ ]:
import streamlit as st
import pandas as pd
from datetime import datetime
import os

file_name='phone_directory.xlsx'

def init_excel():
    if not os.path.exists(file_name):
        contacts_df=pd.DataFrame(columns=['Name','Phone'])
        deleted_df=pd.DataFrame(columns=['Name','Phone','Deleted_on'])

        with pd.ExcelWriter(file_name,engine='openpyxl') as writer:
            contacts_df.to_excel(writer,sheet_name='contacts',index=False)
            deleted_df.to_excel(writer,sheet_name='Deleted_contacts',index=False)

def load_contacts():
    return pd.read_excel(file_name,sheet_name="contacts")

def load_deleted_contacts():
    return pd.read_excel(file_name,sheet_name='Deleted_contacts')

def save_contacts(df):
    with pd.ExcelWriter(file_name,engine='openpyxl',mode='a',
                        if_sheet_exists='replace') as writer:
        df.to_excel(writer,sheet_name='Deleted_contacts',index=False)

def valid_phone(phone):
    return phone.isdigit() and 7 <= len(phone)<=15

st.set_page_config(page_title="phone directory",layout='centered')
st.title("Phone Directory Manager")

init_excel()

menu=st.sidebar.radio(
    "Menu",
    ["Add contact","Search contact","Delete contact","View All contacts","view deleted contacts"])

contact_df=load_contacts()
deleted_df=load_deleted_contacts()


if menu == "Add contact":
    st.subheader(" Add New contact")
    name=st.text_input("Contact Name")
    phone=st.text_input("Phone Number")

    if st.button("save contact"):
        if not name or not phone:
            st.warning("please enter both name and phone number.")

        elif not valid_phone(phone):
            st.error("phone number must contain only digits (7-15 digits).")

        elif name in contact_df['Name'].values:
            st.error("contact name already exists")

        elif phone in contact_df['phone'].values:
            st.error("Phone number already exists")

        else:
            new_row=pd.DataFrame({'Name':[name],'Phone':[phone]})
            contact_df=pd.concat([contact_df,new_row],ignore_index=True)
            save_contacts(contact_df)
            st.success("Contact added successfully")

elif menu=="Search contact":
    st.subheader("search contact")
    search_term=st.text_input("Enter Name or Phone")
    if st.button("Search"):
        if search_item:
            result=contact_df[
            (contact_df['Name'].str.contains(search_term,case=False,na=False))|
            (contact_df['phone'].astype(str).str.contains(search_term))
            ]
            if not result.empty:
                st.success(f"{len(result)} contact(s) found:")
                st.dataframe(result)
            else:
                st.error("No contact found")
        else:
            st.warnining("please enter search term")

elif menu=="Delete contact":
    st.subheader("Delete contact")

    delete_name=st.selectbox("Select contact to delete",contact_df['Name'])
    if st.button("Delete"):
        if delete_name:
            deleted_contact=contact_df[contact_df['Name']==delete_name]

        if not deleted_contact.empty:
            deleted_contact['Deleted_ON']=datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            contact_df=contact_df[contact_df['Name']!=delete_name]

            deleted_df=pd.concat([deleted_df,deleted_contact],ignore_index=True)

            save_contacts(contact_df)
            save_deleted_contacts(deleted_df)

            st.success("Contact deleted successfully!")

elif menu=="View All contacts":
    st.subheader("All contacts")
    if contact_df.empty:
        st.info("No contacts available")
    else:
        st.dataframe(contact_df)
        st.write(f"Total Contacts:{len(contact_df)}")

elif menu=="view deleted contacts":
    st.subheader("deleted contacts history")

    if deleted_df.empty:
        st.info("no deleted contacts.")
    else:
        st.dataframe(deleted_df)
        st.write(f"Total Deleted Contacts:{len(deleted_df)}")

2026-02-21 13:24:43.202 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 13:24:43.203 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 13:24:43.309 
  command:

    streamlit run /home/gopal/takeitsmart/.venv/lib/python3.14/site-packages/ipykernel_launcher.py [ARGUMENTS]
2026-02-21 13:24:43.310 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 13:24:43.310 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


ModuleNotFoundError: No module named 'openpyxl'